# IdP-proxyを学認へ申請する

---
IdP-proxyをSPとして学認へ申請する

## はじめに

CoursewareHubの構成要素を以下に示します。

![構成](images/cw-221-01.png)

このNotebookでは`IdP-proxy` と学認との連携に必要となる申請や、登録などについて記します。

## 学認にSP設置の申請を行う

申請を行う前に学認（GakuNin）の「[参加情報](https://www.gakunin.jp/join)」にてフェデレーション参加の流れを確認してください。参加するフェデレーションに従い「[テストフェデレーション参加手続き](https://www.gakunin.jp/join/test)」または「[運用フェデレーション参加手続き](https://www.gakunin.jp/join/production)」にある「学認申請システム」から「新規SP申請」を行います。

「新規SP申請」を選択すると以下のような画面が表示されます。
> キャプチャー画面はテストフェデレーションのものです。

![新規SP申請](images/cw-531-01.png)

[IdP-proxy](https://github.com/NII-cloud-operation/CoursewareHub-LC_idp-proxy)では[SimpleSAMLphp](https://simplesamlphp.org/)を利用しています。そのため学認申請システムのテンプレートではなく、構築した IdP-proxyのメタデータを「テンプレート外メタデータ」からアップロードして申請してください。

![メタデータアップロード](images/cw-531-02.png)

申請システムにメタデータをアップロードするために、構築したIdP-proxyからメタデータのダウンロードを行います。次のセルで対象となるIdP-proxyのホスト名(FQDN)を指定してください。

In [ ]:
# (例)
# auth_fqdn = 'idpproxy.example.org'

auth_fqdn = 

IdP-proxyからメタデータをダウンロードします。次のセルを実行することで表示されるリンク先にアクセスするとIdP-proxyのメタデータがダウンロードされます。

In [ ]:
print(f'https://{auth_fqdn}/simplesaml/module.php/saml/sp/metadata.php/default-sp')

ダウンロードしたメタデータを学認申請システムにアップロードすると「SPメタデータ情報」の入力欄のうちメタデータに対応するものが設定されます。他の欄を入力した後に申請してください。ただし CoursewareHubでは `mail` 属性を利用するので「受信する属性情報」に `mail` を**必須** な項目として追加してください。

## 学認mAPとの連携

CoursewareHubでは利用者グループを管理するために[学認mAP](https://meatwiki.nii.ac.jp/confluence/display/gakuninmappublic/Home)を利用します。ここでは IdP-proxyを学認mAPと連携する手順について記します。

### 申請

学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通して、構築したSP(IdP-proxy)と学認mAPとを連携するための依頼を行ってください。

### メタデータの登録（テストフェデレーションの場合）

テストフェデレーションの場合、mAP(SP検証環境)のメタデータをIdP-proxyに登録する必要があります。

mAP利用申請を行うと、学認クラウドゲートウェイサービスサポートからmAP(SP検証環境)のメタデータが送られてきます。送られてきたメタデータを IdP-proxyに登録します。メタデータファイルをこのNotebook環境に配置し、そのパスを次のセルで指定してください。

In [ ]:
# (例)
# cgidp_metadata = './sptestcgidp-metadata.xml'

cgidp_metadata = 

操作対象となるIdP-proxyのAnsibleグループ名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleグループ名を指定してください。

In [ ]:
# (例)
# target_auth = 'IdPproxy'

target_auth = 

SP検証環境のメタデータをIdP-proxyの環境に配置します。

In [ ]:
!ansible {target_auth} \
    -b -m copy -a 'src={cgidp_metadata} backup=yes \
    dest={{{{idp_proxy_metadata_dir}}}}/cgidp-metadata.xml'

SimpleSAMLphpの設定ページでSP検証環境のメタデータが登録されたことを確認します。次のセルを実行すると表示されるリンク先をアクセスしてください。

In [ ]:
print(f'https://{auth_fqdn}/simplesaml/module.php/core/frontpage_federation.php')

管理者でログインを行い「SAML 2.0 IdPメタデータ」に「GakuNin Cloud Gateway Service (SP Test) (IdP)」が追加されていることを確認してください。

> SimpleSAMLphpの管理者パスワードは「321-CoursewareHubのセットアップ.ipynb」の「6 SimpleSAMLphpの設定ファイルを更新する」にて `config.php` の `auth.adminpassword` に設定した値となります。

### SPコネクタの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)またはSP検証環境(テストフェデレーションの場合)にアクセスしてSPコネクタの作成を行います。手順の詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタを作成する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873902)」の章を参照してください。

> テストフェデレーションSP検証環境のアドレスは利用申請後にメールなどで通知されます。

SP管理者が学認クラウドゲートウェイのウェブサイトにアクセスすると以下のような画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![クラウドゲートウェイ画面](images/cw-531-03.png)

「SPコネクタの作成」を選択すると入力画面が表示されるので、必要な項目を入力してください。

![新規SPコネクタ入力画面](images/cw-531-04.png)

詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」との項目で「検索対象としない」を選択した場合は、グループ管理者がSPコネクタを見つけることが出来なくなります。グループとの接続を行うためにはSPコネクタ側からグループへ接続申請をする必要があります。詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタにグループを接続する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873907)」を参照してください。